In [34]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import joblib

In [12]:
method_algo = 'mfcc'
model_algo = 'perceptron'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [13]:
def build_model():
    # build the network architecture
    model = keras.Sequential([
        # input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

        # 1st hidden layer
        keras.layers.Dense(512, activation='relu'),

        # 2nd hidden layer
        keras.layers.Dense(256, activation='relu'),

        # 3rd hidden layer
        keras.layers.Dense(128, activation='relu'),

        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=optimizer,
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

    return model

In [29]:
# SPLIT DOS DADOS
random_state = 42


X_train, X_test, y_train, y_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                                          y_train,
                                                                          test_size=0.2,
                                                                          stratify=y_train,
                                                                          random_state=random_state)


In [24]:
param_grid = {}

In [ ]:
model = KerasClassifier(build_fn=build_model, epochs=2000, batch_size=128, verbose=1, )

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)

grid_result = grid.fit(inputs_train, targets_train, validation_data=(inputs_valid, targets_valid))

In [37]:
# # print best parameter after tuning
model = grid
best_params = model.best_params_
sampling_rate = n_rate
# TESTA ACCURÁCIAS

score_test = model.score(X_test, y_test)
score_valid = model.score(X_valid, y_valid)
score_train = model.score(X_train, y_train)

y_hat = model.predict(X_test)

# SALVA MODELO
# filename = f'models/gridperceptron/{method_algo}_{f1_score(y_hat, y_test, average="macro")}_{sampling_rate}/acc{score_test}_seed{random_state}.sav'

# Directory.create_directory(filename, file=True)

# joblib.dump(model, filename)

# SALVA ACURÁCIAS E PARAMETROS
dump_info = {
    'method': 'Grid Search Random Forest',
    'seed': random_state,
    'feature_method': method_algo,
    'sample_rate': sampling_rate,
    'train_test': [len(X_train), len(X_valid), len(X_test)],
    'score_train': score_train,
    'score_valid': score_valid,
    'score_test': score_test,
    'f1_micro': f1_score(y_hat, y_test, average='micro'),
    'f1_macro': f1_score(y_hat, y_test, average='macro'),
    'model_file': f'acc{score_test}_seed{random_state}.sav',
    'params': model.best_params_,
    'cv_results': model.cv_results_
}

JSON.create_json_file(f'models/gridperceptron/{method_algo}_{f1_score(y_hat, y_test, average="macro")}_{sampling_rate}/info.json', dump_info)

{}
 1/28 [>.............................] - ETA: 1s

/home/hugo/projeto_pesquisa_som/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


28/28 [==============================] - 0s 13ms/step


In [7]:
# SALVA A ESTRUTURA DO MODELO

# timestamp = int(time.time())

# Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

# JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

# model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# # DECIDE QUANDO PARAR
# earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# # SALVA OS PESOS
# mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
# model_save_filename, monitor="val_accuracy", save_best_only=True
# )

In [8]:
# TREINA O MODELO
# history = model.fit(inputs_train, targets_train,
#             validation_data=(inputs_valid, targets_valid),
#             epochs=10000,
#             batch_size=128,
#             callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 5s 44ms/step - loss: 170.7413 - accuracy: 0.0129 - val_loss: 10.5189 - val_accuracy: 0.0092
Epoch 2/10000
89/89 [==============================] - 2s 21ms/step - loss: 6.2468 - accuracy: 0.0080 - val_loss: 4.8206 - val_accuracy: 0.0050
Epoch 3/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.7218 - accuracy: 0.0091 - val_loss: 4.7815 - val_accuracy: 0.0110
Epoch 4/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6891 - accuracy: 0.0110 - val_loss: 4.7685 - val_accuracy: 0.0110
Epoch 5/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6740 - accuracy: 0.0107 - val_loss: 4.7655 - val_accuracy: 0.0060
Epoch 6/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6580 - accuracy: 0.0089 - val_loss: 4.7685 - val_accuracy: 0.0127
Epoch 7/10000
89/89 [==============================] - 2s 21ms/step - loss: 4.6599 - accuracy: 0.0156 - val_loss: 4.7750 - val_accura

89/89 [==============================] - 2s 21ms/step - loss: 0.4538 - accuracy: 0.8746 - val_loss: 3.8803 - val_accuracy: 0.5745
Epoch 114/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.4493 - accuracy: 0.8799 - val_loss: 4.1687 - val_accuracy: 0.5780
Epoch 115/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.4361 - accuracy: 0.8793 - val_loss: 3.8110 - val_accuracy: 0.5900
Epoch 116/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.4170 - accuracy: 0.8903 - val_loss: 3.6207 - val_accuracy: 0.6006
Epoch 117/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.3892 - accuracy: 0.8980 - val_loss: 4.0190 - val_accuracy: 0.5950
Epoch 118/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.3878 - accuracy: 0.8958 - val_loss: 3.9157 - val_accuracy: 0.6003
Epoch 119/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.3862 - accuracy: 0.8950 - val_loss: 4.1787 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0352 - accuracy: 0.9918 - val_loss: 3.7455 - val_accuracy: 0.7665
Epoch 226/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0247 - accuracy: 0.9941 - val_loss: 3.6487 - val_accuracy: 0.7609
Epoch 227/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0316 - accuracy: 0.9932 - val_loss: 4.2249 - val_accuracy: 0.7428
Epoch 228/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.1134 - accuracy: 0.9787 - val_loss: 4.0408 - val_accuracy: 0.7496
Epoch 229/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0694 - accuracy: 0.9847 - val_loss: 3.8970 - val_accuracy: 0.7577
Epoch 230/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0395 - accuracy: 0.9904 - val_loss: 3.8365 - val_accuracy: 0.7595
Epoch 231/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0365 - accuracy: 0.9916 - val_loss: 3.8471 - val_accuracy: 0

89/89 [==============================] - 2s 21ms/step - loss: 0.0167 - accuracy: 0.9965 - val_loss: 3.4493 - val_accuracy: 0.8115
Epoch 338/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0142 - accuracy: 0.9966 - val_loss: 3.4696 - val_accuracy: 0.8185
Epoch 339/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0116 - accuracy: 0.9971 - val_loss: 3.6715 - val_accuracy: 0.8154
Epoch 340/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0200 - accuracy: 0.9956 - val_loss: 4.4560 - val_accuracy: 0.7980
Epoch 341/10000
89/89 [==============================] - 2s 21ms/step - loss: 0.0426 - accuracy: 0.9932 - val_loss: 4.3766 - val_accuracy: 0.7828
Epoch 342/10000
76/89 [========================>.....] - ETA: 0s - loss: 0.0564 - accuracy: 0.9895

KeyboardInterrupt: 

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                   f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')